In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    """
    Simple NN implementing R^(28*28)->R^10, x -> A(RelU(Bx+b))+c
    """
    def __init__(self):
        super(Net, self).__init__()
        # define our two linear layers
        self.fc1 = nn.Linear(1*28*28,100)
        self.fc2 = nn.Linear(100,10)
    
    def forward(self, x):
        # implement the forward pass
        z = self.fc1(x)
        x = F.relu(z)
        x = self.fc2(x)
        return x

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # move data (images) and targets (labels) to desired device
        data, target = data.to(device), target.to(device)
        
        # set gradients to zero
        optimizer.zero_grad()
        
        # forward pass - making sure we have to correct shape
        output = model(data.view(-1,28*28))
        
        # compute CE loss
        loss = F.cross_entropy(output, target)
        
        # compute gradients w.r.t. loss
        loss.backward()
        
        # update parameters
        optimizer.step()
        
        # some logging
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(model, device, test_loader):
    
    model.eval()
    
    test_loss = 0 # track testing loss
    correct = 0   # track correctly classified samples
    
    # we do not need gradients during the evaluation
    with torch.no_grad():
        
        # iterate, in mini-batches, over the testing data
        for data, target in test_loader:
            
            # shift data and targets to the desired device
            data, target = data.to(device), target.to(device)
            
            # forward pass
            output = model(data.view(-1,28*28))
            
            # compute loss and accumulate 
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            # get label from the predictions
            pred = output.argmax(dim=1, keepdim=True)
            
            # count correctly classified samples and accumulate
            correct += pred.eq(target.view_as(pred)).sum().item()

    # average testing loss
    test_loss /= len(test_loader.dataset)

    # print avg. testing loss and accuracy (i.e., correctly classified / num.-samples)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
torch.manual_seed(1234)

device = 'cpu'

# data loader for MNIST (training)
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

# data loader for MNIST (testing)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=False)

# instantiate model
model = Net().to(device)

# use SGD as optimizer, set learning rate (lr) and momentum
optimizer = optim.SGD(
    model.parameters(), 
    lr=0.01, 
    momentum=0.9)

# iterate N times over all data
for epoch in range(1,10+1):
    
    # train
    train(model, device, train_loader, optimizer, epoch)
    
    # evaluate after each epoch is done
    test(model, device, test_loader)
